<a href="https://colab.research.google.com/github/retico/cmepda_medphys/blob/master/L1_code/Lecture1_demo2_read_DICOM_dir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read a DICOMDIR and display the image

DICOMDIR is a special DICOM file usually found in the root folder of the CD/DVD media with DICOM studies. This file includes paths to the DICOM files found on the media.

In [ ]:
%matplotlib inline

## Reading the dataset from Google Drive
Prior to this operation be sure to have added the shared folder to your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/My Drive/CMEPDA_MedPhys_datasets"

In [ ]:
!ls "/content/drive/My Drive/CMEPDA_MedPhys_datasets/IMAGES/DICOM_Examples"

In [ ]:
!cp "/content/drive/My Drive/CMEPDA_MedPhys_datasets/IMAGES/DICOM_Examples/Lung_CT_cd2.zip" /content/

In [ ]:
!ls /content/

In [ ]:
!unzip /content/Lung_CT_cd2.zip

In [ ]:
!ls /content/Lung_CT_cd2/

## Reading a DICOMDIR dataset using the pydicom package

This example illustrates how to use the pydicom package to manage a DICOMDIR dataset,
print some dataset information (metadata), and view the images on different planes using matplotlib.

The pydicom [FileSet](https://pydicom.github.io/pydicom/stable/reference/generated/pydicom.fileset.FileSet.html#pydicom.fileset.FileSet)
and [dcmread](https://pydicom.github.io/pydicom/dev/reference/generated/pydicom.filereader.dcmread.html) will be used

In [ ]:
!pip install pydicom

In [ ]:
import os
from pydicom import dcmread
from pydicom.fileset import FileSet
import numpy as np

In [ ]:
DATASETS = "/content/Lung_CT_cd2"
filename = os.path.join(DATASETS, "dicomdir")
data_dir = os.path.join(DATASETS, "12031814")

In [ ]:
fs = FileSet(filename)

In [ ]:
print(fs)

In [ ]:
# List all files in the directory
#dicom_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".dcm")]
dicom_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
#dicom_files
len(dicom_files)

In [ ]:
# Target SeriesNumber
target_series_number = 5

In [ ]:
# Filter files by SeriesNumber
filtered_files = []
for file in dicom_files:
    ds = dcmread(file)
    if ds.SeriesNumber == target_series_number:
        filtered_files.append(ds)

if not filtered_files:
    raise ValueError(f"No files found for SeriesNumber: {target_series_number}")

In [ ]:
# Sort files by InstanceNumber
filtered_files.sort(key=lambda ds: ds.InstanceNumber)

In [ ]:
filtered_files[0].SeriesNumber, filtered_files[0].SeriesDescription, len(filtered_files)

In [ ]:
volume = np.stack([ds.pixel_array for ds in filtered_files])

print("Volume shape:", volume.shape)  # Example: (number_of_slices, height, width)

In [ ]:
# Apply rescaling if necessary
if hasattr(filtered_files[0], "RescaleSlope") and hasattr(filtered_files[0], "RescaleIntercept"):
    slope = filtered_files[0].RescaleSlope
    intercept = filtered_files[0].RescaleIntercept
    volume = volume * slope + intercept

print(f"Slope = {slope}, Intercept = {intercept} and Rescaled volume shape = {volume.shape}")

## Plot  axial, sagittal and colonal views

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x, y, z = *filtered_files[0].PixelSpacing, filtered_files[0].SliceThickness
x,y,z

In [ ]:
aspect_ratio = {
    'axial': y/x,
    'sagittal': z/y,
    'coronal': z/x
}

In [ ]:
# plot 3 orthogonal slices
a1 = plt.subplot(1, 3, 1)
plt.imshow(volume[volume.shape[0]//2,:, :], cmap='gray')
a1.set_aspect(aspect_ratio['axial'])
a1.axis('off')
a1.set_title('Axial')

a2 = plt.subplot(1, 3, 2)
ct = volume[ :, :, volume.shape[2]//2].T
ct = np.flipud(ct)
ct = np.rot90(ct)
plt.imshow(ct, cmap='gray')
a2.set_aspect(aspect_ratio['sagittal'])
a2.axis('off')
a2.set_title('Sagittal')


a3 = plt.subplot(1, 3, 3)
ct = volume[:, volume.shape[1]//2 , :]
ct = np.flipud(ct)
#ct = np.rot90(ct)
plt.imshow(ct, cmap='gray')
a3.set_aspect(aspect_ratio['coronal'])
a3.axis('off')
a3.set_title('Coronal');
